## Markov model

In [ ]:
import pandas as pd 
import random
from collections import Counter
from scipy.stats import pareto
import scipy.stats as stats
from scipy.stats import norm, kurtosis
random.seed(1217)
F_dataset = pd.read_json('path', orient='records')
def create_powerlaw_p(F_dataset, pareto_alpha):
    #create p
    all_facts = F_dataset['fact'].tolist()
    new_facts = []
    for fact in all_facts:
        reps = pareto.rvs(b=pareto_alpha, scale=1)
        reps = int(np.floor(reps))
        new_facts.extend([fact] * reps)
    return new_facts

def create_uniform_p(F_dataset):
    #create p, which is already uniform
    new_facts = F_dataset['fact'].tolist()
    return new_facts

def sample(new_facts, size):
    #sample with replacement
    training_data = random.sample(new_facts, k=size)
    return training_data

def mono_calc(new_facts):
    fact_counts = Counter(new_facts)
    num_mono = sum(1 for count in fact_counts.values() if count == 1)
    mono_pct = num_mono / len(new_facts)
    return mono_pct
    
class MovieFactMarkovChain:
    def __init__(self, order):
        self.order = order
        self.transitions = defaultdict(lambda: defaultdict(float))
        self.initial_counts = defaultdict(float)
        self.starts = []
        self.EOS = '<EOS>'
        
    def tokenize_fact(self, fact):
        # Modify this method if city facts have a specific format
        return fact.strip().split()
    
    def calculate_log_probability(self, fact):
        """Calculate log probability for a fact using order-n transitions."""
        tokens = self.tokenize_fact(fact) + [self.EOS]
        if len(tokens) < self.order:
            return np.log(1e-10)
        
        # Start with initial probability in log-space
        start_tokens = tuple(tokens[:self.order])
        log_probability = np.log(self.initial_probs.get(start_tokens, 1e-10))
        
        # Use order-n transitions
        for i in range(len(tokens) - self.order):
            current = tuple(tokens[i:i + self.order])
            next_token = tokens[i + self.order]
            
            # Get transition probability
            trans_prob = self.transitions.get(current, {}).get(next_token, 1e-10)
            log_probability += np.log(trans_prob)
        
        return log_probability
    
    def train(self, facts):
        # print(f"\nTraining {self.order}-order Markov Chain...")
        num_facts = len(facts)
        start_time = time.time()
        
        # First pass: collect transitions
        # print("Collecting transitions...")
        for fact in tqdm(facts, total=num_facts):
            tokens = self.tokenize_fact(fact) + [self.EOS]
            if len(tokens) < self.order + 1:
                continue  # Skip if not enough tokens
            
            start_tokens = tuple(tokens[:self.order])
            self.starts.append(start_tokens)
            self.initial_counts[start_tokens] += 1
            
            for i in range(len(tokens) - self.order):
                current = tuple(tokens[i:i + self.order])
                next_token = tokens[i + self.order]
                self.transitions[current][next_token] += 1
        
        total_starts = sum(self.initial_counts.values())
        self.initial_probs = {start: count / total_starts for start, count in self.initial_counts.items()}
        
        for current, next_tokens in tqdm(self.transitions.items()):
            total = sum(next_tokens.values())
            for next_token in next_tokens:
                next_tokens[next_token] /= total
                    
        training_time = time.time() - start_time
    
    def generate_facts(self, num_generations, output_path):
        """Generate new facts"""
        generated_facts = []
        # print(f"\nGenerating {num_generations:,} facts...")
        
        with tqdm(total=num_generations) as pbar:
            while len(generated_facts) < num_generations:
                # Pick a random start sequence
                current = random.choices(
                    population=list(self.initial_probs.keys()),
                    weights=list(self.initial_probs.values()),
                    k=1
                )[0]
                result = list(current)
                
                # Generate until EOS token is produced
                while True:
                    if current not in self.transitions:
                        break  # Can't continue from here
                    
                    # Get possible next tokens and their probabilities
                    possible_tokens = list(self.transitions[current].keys())
                    probabilities = list(self.transitions[current].values())
                    
                    # Generate next token
                    next_token = np.random.choice(possible_tokens, p=probabilities)
                    if next_token == self.EOS:
                        break  # End of sequence
                    result.append(next_token)
                    
                    # Update current context
                    current = tuple(result[-self.order:])
                
                # Add generated fact
                generated_fact = ' '.join(result)
                generated_facts.append({
                    'fact': generated_fact,
                    'generated': 1
                })
                pbar.update(1)
            
            # Create DataFrame and save
            generated_df = pd.DataFrame(generated_facts)
            return generated_df

#-----------------------------------------check hallucinations and calibration-----------------------------------------------
def check_hallucinations(generated_data, training_data, F_dataset):
    
    generated_facts = set(generated_data['fact'])
    F_facts = set(F_dataset)  # All true facts
    
    # Hallucinations are generated facts not in F
    hallucinations = list(generated_facts - F_facts)
    hallucination_rate = len(hallucinations) / len(generated_data)
    
    # Also calculate "unseen but true" rate
    training_facts = set(training_data)
    unseen_true = list(generated_facts - training_facts - set(hallucinations))
    unseen_true_rate = len(unseen_true) / len(generated_data)
    
    return {
        'hallucination_rate': hallucination_rate,
        'hallucinated_facts': hallucinations,
        'unseen_true_rate': unseen_true_rate,
        'unseen_true_facts': unseen_true
    }

def create_epsilon_induced_bins(epsilon: float):
   if epsilon < 0:
       raise ValueError("Epsilon must be non-negative")
   
   if epsilon == 0:
       return "finest"
   
   if epsilon >= 1:
       # When epsilon = 1, return single bin for all probabilities
       return [(0, 1)]
   
   bins = []
   i = 0
   
   while True:
       upper = (1-epsilon)**i
       lower = (1-epsilon)**(i+1)
       
       # If lower bound gets very small, make it 0 and make this our last bin
       if lower < 1e-10:
           bins.append((0, upper))
           break
           
       if upper - lower > 1e-10:
           bins.append((lower, upper))
       
       i += 1
   ##append final bin for edge case where everything is 1
   bins.append((1.0, 1.0))
   return bins
    
def calculate_kalai_calibration_log_binning(F_dataset, training, model, epsilon, output_path):  
    # Define training facts from O_dataset and get frequencies
    all_facts = F_dataset
    fact_counts = Counter(all_facts)
    total_facts = len(all_facts)
    
    # print(f"\nCalculating g(y) probabilities for {total_facts} training facts...")
    
    facts_list = []
    g_prob_list = []
    p_prob_list = []
    
    raw_g_prob_list = []
    raw_p_prob_list = []
    
    for fact in tqdm(all_facts):
        log_prob = model.calculate_log_probability(fact)
        g_prob = np.exp(log_prob)

        facts_list.append(fact)
        g_prob_list.append(g_prob)
        raw_g_prob_list.append(g_prob)
        
        p_prob = fact_counts[fact] / total_facts
        p_prob_list.append(p_prob)
        raw_p_prob_list.append(p_prob)

    # Normalize g(y) and p(y)
    g_sum = sum(g_prob_list)
    if g_sum > 0:
        g_prob_list = [p / g_sum for p in g_prob_list]
    p_sum = sum(p_prob_list)
    if p_sum > 0:
        p_prob_list = [p / p_sum for p in p_prob_list]

    # Create bins
    bins = create_epsilon_induced_bins(epsilon)
    if isinstance(bins, str) and bins == "finest":
        unique_probs = sorted(set(g_prob_list), reverse=True)
        bins = [(p, p) for p in unique_probs]
    
    # ----------------------------------------------------------------
    # 1) Create data structures to accumulate bin memberships + sums.
    # ----------------------------------------------------------------
    binned_facts = [[] for _ in range(len(bins))]

    # Keep track of the sum of p(y) and g(y) in each bin
    binned_p_sums = [0.0] * len(bins)
    binned_g_sums = [0.0] * len(bins)
    
    # -------------------------------------------------------------
    # 2) Assign each fact to exactly one bin and update running sums.
    # -------------------------------------------------------------
    for i, (fact, g_y) in enumerate(zip(facts_list, g_prob_list)):
        assigned = False
        for bin_idx, (lower, upper) in enumerate(bins):
            # If it falls into [lower, upper)
            if lower <= g_y < upper:
                binned_facts[bin_idx].append((fact, g_y))
                binned_p_sums[bin_idx] += p_prob_list[i]
                binned_g_sums[bin_idx] += g_prob_list[i]
                assigned = True
                break
        # If we never broke out, it goes to the last bin
        # (Handles edge-case if upper=1 for last bin.)
        if not assigned:
            last_idx = len(bins) - 1
            binned_facts[last_idx].append((fact, g_y))
            binned_p_sums[last_idx] += p_prob_list[i]
            binned_g_sums[last_idx] += g_prob_list[i]
    
    # ---------------------------------------
    # 3) Calculate miscalibration efficiently
    # ---------------------------------------
    miscalibration = 0.0
    for bin_idx in range(len(bins)):
        p_B = binned_p_sums[bin_idx]
        g_B = binned_g_sums[bin_idx]
        miscalibration += abs(p_B - g_B)
    
    miscalibration *= 0.5
    
    # Calculate log loss
    log_loss = 0
    for fact in training:
        g_prob_train = np.exp(model.calculate_log_probability(fact))
        if g_prob_train > 0:
            log_loss += np.log(1/g_prob_train)
    
    return {
        'miscalibration': float(miscalibration),
        'num_bins': len(binned_facts),
        'log_loss': float(log_loss),
    }
      

## Analysis

In [ ]:
import numpy as np
import os
from collections import defaultdict
import time
from tqdm import tqdm

random.seed(140)

uniform_p = pd.read_json('path', orient='records')
uniform_p = uniform_p[1:20000]
print(uniform_p)
set_order = 2
sample_size = 5000
generation_size = 5000
bin_num = 25
epsilon = 0.1

hall_rates = []
unseen_true_rates = []
pareto_alphas = []
mono_pcts = []
miscals = []
lengths_train = []
entropies = []

max_train_repeats  = []
average_train_repeats = []
twenty_fifths = []
medians = []
seventy_fifths = []
num_uniques = []

max_repeats_p  = []
average_repeats_p = []
twenty_fifths_p = []
medians_p = []
seventy_fifths_p = []
num_uniques_p = []


mono_pcts_p = []
mono_miscal_ds = []

alphas = np.arange(1, 4, 0.5).tolist()
for pareto_alpha in alphas:
    print(f"\nProcessing pareto_alpha: {pareto_alpha}")
    # p distribution
    
    powerlaw_p = create_powerlaw_p(uniform_p, pareto_alpha)
    training_data = sample(powerlaw_p, sample_size)

    # check average and max repeats in training data
    facts_counter = Counter(training_data)
    repeats = list(facts_counter.values())
    max_repeat = max(repeats)
    avg_repeat = sum(repeats) / len(facts_counter)
    twenty_fifth = np.percentile(repeats, 25)
    median = np.percentile(repeats, 50)
    seventy_fifth = np.percentile(repeats, 75)
    num_unique = len(facts_counter.keys())
    # mono rate calc
    mono_rate = mono_calc(training_data)


    #check average and max repeats in p 
    facts_counter_p = Counter(powerlaw_p)
    repeats_p = list(facts_counter_p.values())
    max_repeat_p = max(repeats_p)
    avg_repeat_p = sum(repeats_p) / len(facts_counter_p)
    twenty_fifth_p = np.percentile(repeats_p, 25)
    median_p = np.percentile(repeats_p, 50)
    seventy_fifth_p = np.percentile(repeats_p, 75)
    num_unique_p = len(facts_counter_p.keys())
    mono_rate_p = mono_calc(powerlaw_p)
    
    # fit model
    model = MovieFactMarkovChain(order=set_order)
    model.train(training_data)
    
    #generation path for fact checking
    gen_path = f"path"
    os.makedirs(os.path.dirname(gen_path), exist_ok=True)
    generated_data = model.generate_facts(generation_size, gen_path)
    #hallucination results
    hall_results = check_hallucinations(generated_data, training_data, powerlaw_p)

    #miscalibration results
    miscal_path = f"path"
    miscal_results = calculate_kalai_calibration_log_binning(powerlaw_p, model, epsilon, miscal_path)

    mono_miscal_d = (abs(mono_rate - miscal_results['miscalibration']) * 100) / (mono_rate * 100)
    mono_miscal_ds.append(mono_miscal_d)
    #save results
    mono_pcts.append(mono_rate)
    mono_pcts_p.append(mono_rate_p)
    
    hall_rates.append(hall_results['hallucination_rate'])
    unseen_true_rates.append(hall_results['unseen_true_rate'])
    miscals.append(miscal_results['miscalibration'])
    lengths_train.append(len(training_data))
    pareto_alphas.append(pareto_alpha)
    ## training repeats distribution
    max_train_repeats.append(max_repeat)
    average_train_repeats.append(avg_repeat)
    twenty_fifths.append(twenty_fifth)
    medians.append(median)
    seventy_fifths.append(seventy_fifth)
    num_uniques.append(num_unique)
    ## p repeats distribution
    max_repeats_p.append(max_repeat_p)
    average_repeats_p.append(avg_repeat_p)
    twenty_fifths_p.append(twenty_fifth_p)
    medians_p.append(median_p)
    seventy_fifths_p.append(seventy_fifth_p)
    num_uniques_p.append(num_unique_p)  
    
results_df= pd.DataFrame({
    "Pareto": pareto_alphas,
    'Mono_Pct': mono_pcts,
    'Mono_Pct_P': mono_pcts_p,
    'Mono_Miscal_D': mono_miscal_ds, 
    "Miscalibration": miscals,
    "Hallucination": hall_rates,
    "Unseen_True_Rates": unseen_true_rates,
    "Num_Training_Facts": lengths_train,
    "Max_Train_Repeats": max_train_repeats,
    "Avg_Train_Repeats": average_train_repeats,
    "25_Train_Repeats": twenty_fifths,
    "50_Train_Repeats": medians,
    "70_Train_Repeats": seventy_fifths,
    "Max_P_Repeats": max_repeats_p,
    "Avg_P_Repeats": average_repeats_p,
    "25_P_Repeats": twenty_fifths_p,
    "50_P_Repeats": medians_p,
    "70_P_Repeats": seventy_fifths_p,
    "Num_Unique_Train": num_uniques, 
    "Num_Unique_P": num_uniques_p
    })

# Print summary statistics
results_df.to_csv("path")

## Model Capacity Analysis

In [ ]:
import numpy as np
import os
from collections import defaultdict
import time
from scipy.stats import kurtosis
from tqdm import tqdm

random.seed(140)

uniform_p = pd.read_json('path', orient='records')
uniform_p = uniform_p[1:20000]
print(uniform_p)
set_orders = [1, 2, 3, 4, 5, 6]
sample_size = 5000
generation_size = 5000
bin_num = 25
epsilon = 0.1

hall_rates = []
unseen_true_rates = []
pareto_alphas = []
mono_pcts = []
miscals = []
lengths_train = []
entropies = []

max_train_repeats  = []
average_train_repeats = []
twenty_fifths = []
medians = []
seventy_fifths = []
num_uniques = []

max_repeats_p  = []
average_repeats_p = []
twenty_fifths_p = []
medians_p = []
seventy_fifths_p = []
num_uniques_p = []
mono_pcts_p = []

capacities = []

alphas = np.arange(1, 4, 0.025).tolist()
for pareto_alpha in alphas:
    print(f"\nProcessing pareto_alpha: {pareto_alpha}")
    # p distribution
    for set_order in set_orders:
        powerlaw_p = create_powerlaw_p(uniform_p, pareto_alpha)
        training_data = sample(powerlaw_p, sample_size)

        capacities.append(set_order)
        # check average and max repeats in training data
        facts_counter = Counter(training_data)
        repeats = list(facts_counter.values())
        max_repeat = max(repeats)
        avg_repeat = sum(repeats) / len(facts_counter)
        twenty_fifth = np.percentile(repeats, 25)
        median = np.percentile(repeats, 50)
        seventy_fifth = np.percentile(repeats, 75)
        num_unique = len(facts_counter.keys())
        # mono rate calc
        mono_rate = mono_calc(training_data)

    
        #check average and max repeats in p 
        facts_counter_p = Counter(powerlaw_p)
        repeats_p = list(facts_counter_p.values())
        max_repeat_p = max(repeats_p)
        avg_repeat_p = sum(repeats_p) / len(facts_counter_p)
        twenty_fifth_p = np.percentile(repeats_p, 25)
        median_p = np.percentile(repeats_p, 50)
        seventy_fifth_p = np.percentile(repeats_p, 75)
        num_unique_p = len(facts_counter_p.keys())
        mono_rate_p = mono_calc(powerlaw_p)
        
        # fit model
        model = MovieFactMarkovChain(order=set_order)
        model.train(training_data)
        
        #generation path for fact checking
        gen_path = f"path"
        os.makedirs(os.path.dirname(gen_path), exist_ok=True)

        generated_data = model.generate_facts(generation_size, gen_path)
        #hallucination results
        hall_results = check_hallucinations(generated_data, training_data, powerlaw_p)
    
        #miscalibration results
        miscal_path = f"path"
        miscal_results = calculate_kalai_calibration_log_binning(powerlaw_p, training_data, model, epsilon, miscal_path)
    
        #save results
        mono_pcts.append(mono_rate)
        mono_pcts_p.append(mono_rate_p)
        
        hall_rates.append(hall_results['hallucination_rate'])
        unseen_true_rates.append(hall_results['unseen_true_rate'])
        miscals.append(miscal_results['miscalibration'])
        lengths_train.append(len(training_data))
        pareto_alphas.append(pareto_alpha)
        ## training repeats distribution
        max_train_repeats.append(max_repeat)
        average_train_repeats.append(avg_repeat)
        twenty_fifths.append(twenty_fifth)
        medians.append(median)
        seventy_fifths.append(seventy_fifth)
        num_uniques.append(num_unique)
        ## p repeats distribution
        max_repeats_p.append(max_repeat_p)
        average_repeats_p.append(avg_repeat_p)
        twenty_fifths_p.append(twenty_fifth_p)
        medians_p.append(median_p)
        seventy_fifths_p.append(seventy_fifth_p)
        num_uniques_p.append(num_unique_p)  
    
results_df= pd.DataFrame({
    "Pareto": pareto_alphas,
    "Capacity": capacities,
    'Mono_Pct': mono_pcts,
    'Mono_Pct_P': mono_pcts_p,
    "Miscalibration": miscals,
    "Hallucination": hall_rates,
    "Log_Loss": entropies,
    "Unseen_True_Rates": unseen_true_rates,
    "Num_Training_Facts": lengths_train,
    "Max_Train_Repeats": max_train_repeats,
    "Avg_Train_Repeats": average_train_repeats,
    "25_Train_Repeats": twenty_fifths,
    "50_Train_Repeats": medians,
    "70_Train_Repeats": seventy_fifths,
    "Max_P_Repeats": max_repeats_p,
    "Avg_P_Repeats": average_repeats_p,
    "25_P_Repeats": twenty_fifths_p,
    "50_P_Repeats": medians_p,
    "70_P_Repeats": seventy_fifths_p,
    "Num_Unique_Train": num_uniques, 
    "Num_Unique_P": num_uniques_p
    })

results_df.to_csv("path")